# Data Cleaning using Python

The problem that you will encounter in data cleaning are 
missing values,
Column names,
Outliers,
unexpected observations in columns,
Duplicate Rows,
Untidy Data.
We will cover all of them as our data have all these problems.
The data i am using for the demo is UCI Data set repository and can be downloaded from 'https://archive.ics.uci.edu/ml/machine-learning-databases/diabetes/' the data is related to diabetic patients.Complete description of data can be found with data files.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
from datetime import datetime
import re

There are 70 data files so i am using glob function from glob library to read all the file paths in list. 

In [2]:
file_list =glob.glob('Path/To/Your/Directory/Diabetes-Data/data-*')
len(file_list)

70

i use list comprehension to read all the files in a single list of 70 data frames. 

In [3]:
list_data = [pd.read_csv(x,sep='\t',names=['date','time','code','value']) for x in file_list]
len(list_data)

70

Concatinated all the dataframes in a single dataframe

In [4]:
deb_df = pd.concat(list_data,ignore_index=True)
deb_df.shape

(29330, 4)

In [5]:
deb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29330 entries, 0 to 29329
Data columns (total 4 columns):
date     29297 non-null object
time     29330 non-null object
code     29330 non-null int64
value    29297 non-null object
dtypes: int64(1), object(3)
memory usage: 916.6+ KB


# Removing Missing Vlaues

For getting all rows having NaN values in dataframe.

In [6]:
nan_values = deb_df[deb_df.isnull().T.any().T]
print nan_values

             date  time  code value
9960   10-12-1989  7:00     0   NaN
9961          NaN   138    33    3A
9962   10-12-1989  7:00     0   NaN
9963          NaN   006    33    21
9964   10-12-1989  7:00     0   NaN
9965          NaN   016    33    22
11239  05-25-1990  7:00     0   NaN
11240         NaN   201    33    3A
11241  05-25-1990  7:00     0   NaN
11242         NaN   004    33    21
11243  05-25-1990  7:00     0   NaN
11244         NaN   018    33    22
11465  07-02-1990  7:00     0   NaN
11466         NaN   202    33    3A
11467  07-02-1990  7:00     0   NaN
11468         NaN   006    33    21
11469  07-02-1990  7:00     0   NaN
11470         NaN   018    33    22
11563  07-14-1990  7:00     0   NaN
11564         NaN   195    33    3A
11565  07-14-1990  7:00     0   NaN
11566         NaN   006    33    21
11567  07-14-1990  7:00     0   NaN
11568         NaN   018    33    22
11590  07-17-1990  7:00     0   NaN
11591         NaN   168    33    3A
11592  07-17-1990  7:00     

In [7]:
len(nan_values)

66

i am removing missing values from the data as most of them are not making any sense and they are very small in number with respect to the data.

In [8]:
deb_df1 =deb_df.dropna()
deb_df1[deb_df1.isnull().T.any().T]

,date,time,code,value


In [9]:
deb_df1.shape

(29264, 4)

# Removing Duplicates

i am using two methods for finding and removing duplicates but there are few duplicates in data as shape of data is describing.

In [12]:
deb_df1.drop_duplicates(subset=deb_df1.columns,keep='last',inplace=True)
deb_df1.shape

(27438, 4)

# Testing and Removing unexpected Values

Now i had tried to drill down the data for further cleaning of any unexpected value.  

In [13]:
deb_df1[deb_df1.date.str.contains(r'nan')]

,date,time,code,value


In [14]:
deb_df1[deb_df1.date.apply(lambda x:int(x.rsplit('-')[0])>12) | deb_df1.date.apply(lambda x:int(x.rsplit('-')[1])>31)]

,date,time,code,value


We got unexpected values in time column as we were not expecting 56 or 188 hours in time.

In [15]:
deb_df1[deb_df1.time.apply(lambda x:int(x.rpartition(':')[0])>24)]

,date,time,code,value
28070,04-09-1991,56:35,58,237
28071,04-09-1991,56:35,33,16
28072,04-09-1991,56:35,34,40
28171,04-21-1991,188:00,62,128
28172,04-21-1991,188:00,33,14


i am making all of them 00:00.

In [17]:
deb_df1.ix[deb_df1.time.apply(lambda x:int(x.rpartition(':')[0])>24),'time']='00:00'

In [18]:
deb_df1[deb_df1.time.apply(lambda x:x=='00:00')]

,date,time,code,value
22922,02-08-1989,00:00,57,45
25875,02-03-1990,00:00,57,284
25879,02-04-1990,00:00,57,155
25904,02-12-1990,00:00,57,124
26784,08-01-1989,00:00,48,204
28070,04-09-1991,00:00,58,237
28071,04-09-1991,00:00,33,16
28072,04-09-1991,00:00,34,40
28171,04-21-1991,00:00,62,128
28172,04-21-1991,00:00,33,14


we can perform above task by using function.

In [19]:
def timeClean(df,col='time'):
    #print df
    #print df[col]
    for key,val in df.iterrows():
        if int(val[col].rpartition(':')[0])>24:
            df.loc[key,col]='00:00'
    return df
deb_df2 = timeClean(df= deb_df1,col='time')

to check any unexpected value in mins in time column.

In [20]:
deb_df1[deb_df1.time.apply(lambda x:int(x.rpartition(':')[2])>60)]

,date,time,code,value


That's Most trickist one as it was due to data present in wrong date as 6 month does not have 31 days in it.But we get some unexpected values in date so we are changing it to 1st of 7th month.

In [21]:
#date cleaning
pattern = '06-31-\d*'
deb_df1.ix[deb_df1.date.apply(lambda x: bool(re.match(pattern,string=x)))]

,date,time,code,value
6505,06-31-1991,07:50,58,149
6506,06-31-1991,07:50,33,4
6507,06-31-1991,07:50,34,24
6508,06-31-1991,13:30,60,162
6509,06-31-1991,13:30,33,5
6510,06-31-1991,19:45,62,213
6511,06-31-1991,19:45,33,11


In [22]:
deb_df1.ix[deb_df1.date.apply(lambda x: bool(re.match(pattern,string=x))),'date']='07-01-1991'

In [23]:
deb_df1.ix[deb_df1.date.apply(lambda x: bool(re.match(pattern,string=x)))]

,date,time,code,value


Now we will change the types of data and create a new variable period so that we can convert that is date time format.

In [25]:
#before that we have to clean date and time date ='nan' and time >24H time[2]>24
def dfClean(df):
    df.loc[:,'value'] = pd.to_numeric(df['value'],errors = 'coerce')
    df.loc[:,'code'] = df.code.astype('category')
    df.loc[:,'period'] =df["date"].map(str) + " "+df["time"].map(str)
    return df
df = dfClean(deb_df1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27438 entries, 0 to 29329
Data columns (total 5 columns):
date      27438 non-null object
time      27438 non-null object
code      27438 non-null category
value     27430 non-null float64
period    27438 non-null object
dtypes: category(1), float64(1), object(3)
memory usage: 1.1+ MB


Converting the period data type to date time format.

In [26]:
df.loc[:,'period']=df.period.apply(lambda x: datetime.strptime(x, '%m-%d-%Y %H:%M'))

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27438 entries, 0 to 29329
Data columns (total 5 columns):
date      27438 non-null object
time      27438 non-null object
code      27438 non-null category
value     27430 non-null float64
period    27438 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](1), float64(1), object(2)
memory usage: 1.1+ MB


In [28]:
df.head()

,date,time,code,value,period
0,04-21-1991,9:09,58,100.0,1991-04-21 09:09:00
1,04-21-1991,9:09,33,9.0,1991-04-21 09:09:00
2,04-21-1991,9:09,34,13.0,1991-04-21 09:09:00
3,04-21-1991,17:08,62,119.0,1991-04-21 17:08:00
4,04-21-1991,17:08,33,7.0,1991-04-21 17:08:00


As we have period column so we are removing date and time columns.

In [29]:
del df['date']

In [30]:
del df['time']

In [31]:
df.head()

,code,value,period
0,58,100.0,1991-04-21 09:09:00
1,33,9.0,1991-04-21 09:09:00
2,34,13.0,1991-04-21 09:09:00
3,62,119.0,1991-04-21 17:08:00
4,33,7.0,1991-04-21 17:08:00


# Column Names and categories

Importing the text file having categories code.

In [32]:
 with open('path') as file:
        codes = file.readlines()

Filtering all the codes from text file and joining with the original dataframe.

In [33]:
codes_list= codes[20:40]
codes_list

['33 = Regular insulin dose\n',
 '34 = NPH insulin dose\n',
 '35 = UltraLente insulin dose\n',
 '48 = Unspecified blood glucose measurement\n',
 '57 = Unspecified blood glucose measurement\n',
 '58 = Pre-breakfast blood glucose measurement\n',
 '59 = Post-breakfast blood glucose measurement\n',
 '60 = Pre-lunch blood glucose measurement\n',
 '61 = Post-lunch blood glucose measurement\n',
 '62 = Pre-supper blood glucose measurement\n',
 '63 = Post-supper blood glucose measurement\n',
 '64 = Pre-snack blood glucose measurement\n',
 '65 = Hypoglycemic symptoms\n',
 '66 = Typical meal ingestion\n',
 '67 = More-than-usual meal ingestion\n',
 '68 = Less-than-usual meal ingestion\n',
 '69 = Typical exercise activity\n',
 '70 = More-than-usual exercise activity\n',
 '71 = Less-than-usual exercise activity\n',
 '72 = Unspecified special event\n']

In [34]:
cat_code = []
cat_list =[]
for line in codes_list:
    split = line.rsplit('=')
    cat_code.append(split[0][:2])
    cat_list.append(split[1][:-1])

In [35]:
category = pd.DataFrame({'category':cat_list,'code':cat_code})

In [36]:
category['code'] = pd.to_numeric(category['code'],errors = 'coerces')
category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
category    20 non-null object
code        20 non-null int64
dtypes: int64(1), object(1)
memory usage: 392.0+ bytes


In [37]:
dabatic = pd.merge(left =df,right = category,on='code')

Removing code column as we have category column and rearranging the columns.

In [38]:
cols = ['period','category', 'value']
dabatic = dabatic[cols]
dabatic.head()

,period,category,value
0,1991-04-21 09:09:00,Pre-breakfast blood glucose measurement,100.0
1,1991-04-22 07:35:00,Pre-breakfast blood glucose measurement,216.0
2,1991-04-23 07:25:00,Pre-breakfast blood glucose measurement,257.0
3,1991-04-24 07:52:00,Pre-breakfast blood glucose measurement,239.0
4,1991-04-25 07:29:00,Pre-breakfast blood glucose measurement,67.0


# Tidy Data from untidy

Now data is alomost cleaned but its untidy so we have to convert it to tidy. 

In [39]:
dabatic_clean = pd.pivot_table(dabatic,values='value',index=['period'],columns=['category'],aggfunc=np.mean)

In [40]:
dabatic_clean.head()

category,Hypoglycemic symptoms,Less-than-usual exercise activity,Less-than-usual meal ingestion,More-than-usual exercise activity,More-than-usual meal ingestion,NPH insulin dose,Post-breakfast blood glucose measurement,Post-lunch blood glucose measurement,Post-supper blood glucose measurement,Pre-breakfast blood glucose measurement,Pre-lunch blood glucose measurement,Pre-snack blood glucose measurement,Pre-supper blood glucose measurement,Regular insulin dose,Typical exercise activity,Typical meal ingestion,UltraLente insulin dose,Unspecified blood glucose measurement,Unspecified special event
period,,,,,,,,,,,,,,,,,,,
1988-03-27 08:00:00,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988-03-27 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988-03-27 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258.0,NaN,NaN,NaN,NaN,NaN,NaN
1988-03-28 08:00:00,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988-03-28 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Validating Results

In [41]:
dabatic[dabatic.period.apply(lambda x:x==datetime.strptime('03-27-1988 08:00', '%m-%d-%Y %H:%M'))]

,period,category,value
3115,1988-03-27 08:00:00,Pre-breakfast blood glucose measurement,134.0
15523,1988-03-27 08:00:00,NPH insulin dose,20.0


As there are missing values in data set so i am resampling it on monthly basis.

In [42]:
db_df = dabatic_clean.resample('M').mean().interpolate(method='linear').fillna(method= 'bfill')

In [43]:
db_df.head()

category,Hypoglycemic symptoms,Less-than-usual exercise activity,Less-than-usual meal ingestion,More-than-usual exercise activity,More-than-usual meal ingestion,NPH insulin dose,Post-breakfast blood glucose measurement,Post-lunch blood glucose measurement,Post-supper blood glucose measurement,Pre-breakfast blood glucose measurement,Pre-lunch blood glucose measurement,Pre-snack blood glucose measurement,Pre-supper blood glucose measurement,Regular insulin dose,Typical exercise activity,Typical meal ingestion,UltraLente insulin dose,Unspecified blood glucose measurement,Unspecified special event
period,,,,,,,,,,,,,,,,,,,
1988-03-31,0.0,0.0,0.0,0.0,0.0,20.000000,242.0,221.0,259.0,116.333333,156.000000,150.625,180.000000,4.541667,0.0,0.0,14.307692,137.400000,0.0
1988-04-30,0.0,0.0,0.0,0.0,0.0,19.533333,242.0,221.0,259.0,114.533333,119.285714,150.625,135.533333,4.541667,0.0,0.0,14.307692,137.400000,0.0
1988-05-31,0.0,0.0,0.0,0.0,0.0,20.000000,242.0,221.0,259.0,104.133333,118.307692,150.625,144.733333,4.541667,0.0,0.0,14.307692,137.400000,0.0
1988-06-30,0.0,0.0,0.0,0.0,0.0,14.700000,242.0,221.0,259.0,99.766667,95.566667,150.625,139.666667,4.541667,0.0,0.0,14.307692,141.623077,0.0
1988-07-31,0.0,0.0,0.0,0.0,0.0,15.516129,242.0,221.0,259.0,120.435484,121.433333,150.625,155.620690,4.541667,0.0,0.0,14.307692,145.846154,0.0


As few columns only have zero values so we can remove them from the data.Now the final output we have is the cleaned data on which we can perform analysis and model building as data is tidy now.Which was our target.
### Thanks To Pandas